
---
# A Synthetic Text Generator

This section adds an interactive Gradio app that:

- Uses Hugging Face `pipeline`
- Allows model selection (2 open-source models)
- Allows prompt type selection
- Includes creativity control (temperature)
- Runs with GPU if available


In [ ]:
# Pip installs should come at the top line.
# If your Kernel ever resets, you need to run this again.

!pip install -q --upgrade datasets==3.6.0 transformers==4.57.6 gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 23.8 MB/s eta 0:00:00


In [ ]:
# Let's check the GPU - it should be a Tesla T4

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)
  if gpu_info.find('Tesla T4') >= 0:
    print("Success - Connected to a T4")
  else:
    print("NOT CONNECTED TO A T4")

In [ ]:
# Imports

import torch
from google.colab import userdata
from huggingface_hub import login
from transformers import pipeline
import gradio as gr

In [ ]:
hf_token = userdata.get('HF_TOKEN')
if hf_token and hf_token.startswith("hf_"):
  print("HF key looks good so far")
else:
  print("HF key is not set - please click the key in the left sidebar")
login(hf_token, add_to_git_credential=True)

In [ ]:

MODELS = {
    "GPT-2": "gpt2",
    "GEMMA": "google/gemma-3-270m-it"
}

PROMPT_TYPES = {
    "Movie Review": "Write a detailed movie review:",
    "Blog Post": "Write a short blog post about productivity:",
    "Product Review": "Write a product review for a new smartphone:",
    "Customer Complaint": "Write a frustrated customer complaint email:",
    "News Article": "Write a short news article about artificial intelligence:"
}

def generate_text(model_choice, prompt_type, temperature):
    model = MODELS[model_choice]
    generator =  pipeline("text-generation", model=model, device="cuda")
    prompt = PROMPT_TYPES[prompt_type]
    output = generator(prompt, max_length=120, do_sample=True, temperature=temperature)
    return output[0]["generated_text"]


view = gr.Interface(
    fn=generate_text,
    inputs=[
        gr.Dropdown(list(MODELS.keys()), label="Choose Model"),
        gr.Dropdown(list(PROMPT_TYPES.keys()), label="Choose Prompt Type"),
        gr.Slider(0.3, 1.2, value=0.8, step=0.1, label="Creativity (Temperature)")
    ],
    outputs=gr.Markdown(label="Generated Text:"),
    title="Synthetic Text Generator",
    flagging_mode="never"
)

view.launch()